In [1]:
!pip install -qq transformers
!pip install -qq torchflare


     |████████████████████████████████| 2.6 MB 7.6 MB/s 
     |████████████████████████████████| 3.3 MB 23.3 MB/s 
     |████████████████████████████████| 636 kB 50.9 MB/s 
     |████████████████████████████████| 895 kB 41.7 MB/s 
     |████████████████████████████████| 85 kB 3.0 MB/s 
     |████████████████████████████████| 17.3 MB 41 kB/s 
     |████████████████████████████████| 51.0 MB 79 kB/s 
     |████████████████████████████████| 3.0 MB 37.4 MB/s 
     |████████████████████████████████| 61 kB 376 kB/s 
     |████████████████████████████████| 735.5 MB 12 kB/s 
     |████████████████████████████████| 10.3 MB 37.9 MB/s 
     |████████████████████████████████| 22.3 MB 1.6 MB/s 
     |████████████████████████████████| 9.9 MB 24.8 MB/s 
     |████████████████████████████████| 15.3 MB 280 kB/s 
     |████████████████████████████████| 98 kB 7.1 MB/s 
     |████████████████████████████████| 37.1 MB 81 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the pac

In [2]:
import torchflare
import re
import torchflare.callbacks as cbs
import pandas as pd
import transformers
import torch
import sklearn.metrics as skm
from torchflare.experiments import Experiment, ModelConfig
import torchflare.criterion as crit
from torchflare.datasets import TextDataloader
from torchflare.metrics import MetricMeter, _BaseMetric
import pickle

In [3]:
train_df = pd.read_csv("/content/drive/MyDrive/Hahakathon/train.csv")
valid_df = pd.read_csv("/content/drive/MyDrive/Hahakathon/dev.csv")

In [4]:
#Change Backbone name here.
BACKBONE_NAME = "prajjwal1/bert-small"

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained(BACKBONE_NAME)

In [23]:
train_dl = TextDataloader.from_df(
    df=train_df,
    input_col="text",
    label_cols="is_humor",
    tokenizer=tokenizer,
    max_len=256,
).get_loader(batch_size=16, shuffle=True)

valid_dl = TextDataloader.from_df(
    df=valid_df,
    input_col="text",
    label_cols="is_humor",
    tokenizer=tokenizer,
    max_len=256,
).get_loader(batch_size=16, shuffle = False)

In [24]:
class SklearnF1(MetricMeter, _BaseMetric):
    def __init__(self):

        self.f1 = skm.f1_score
        self._outputs = None
        self._targets = None
        self.reset()

    def handle(self):
        return self.f1.__name__.lower()

    def accumulate(self, outputs: torch.Tensor, targets: torch.Tensor):
        """Method to accumulate the outputs and targets.
        Args:
            outputs(torch.Tensor) : raw logits from the network.
            targets(torch.Tensor) : Ground truth targets
        """
        outputs, targets = self.detach_tensor(outputs), self.detach_tensor(targets)
        outputs = torch.argmax(outputs, dim=1)
        # print(outputs)
        self._outputs.append(outputs)
        self._targets.append(targets)

    def reset(self):
        """Resets the accumulation lists."""
        self._outputs = []
        self._targets = []

    @property
    def value(self):

        outputs = torch.cat(self._outputs)
        targets = torch.cat(self._targets)
        f1_score = self.f1(targets.numpy(), outputs.numpy())
        return torch.tensor(f1_score)

In [25]:
class BackBoneModel(torch.nn.Module):
    def __init__(self, out_features, model_path):
        super(BackBoneModel, self).__init__()
        self.model = transformers.AutoModel.from_pretrained(
            model_path, return_dict=False
        )
        in_features = self.model.pooler.dense.out_features
        self.linear = torch.nn.Linear(in_features, out_features)
        #self.dropout = torch.nn.Dropout(0.2)

    def forward(self, x):

        _, o_2 = self.model(**x)
        op = self.linear(o_2)
        return op

In [26]:
@cbs.on_experiment_end(order = cbs.CallbackOrder.EXTERNAL)
def save_pickle_experiment(experiment : "Experiment"):
    if "/" in BACKBONE_NAME:
        prefix = BACKBONE_NAME.split("/")[1]
    else:
        prefix = BACKBONE_NAME
    pickle.dump(experiment.history , open(f"{prefix}-experiment.pkl" , "wb"))

In [27]:
config = ModelConfig(
    nn_module=BackBoneModel,
    module_params={"out_features": 2, "model_path": BACKBONE_NAME},
    optimizer="AdamW",
    optimizer_params={"lr": 2e-5, "weight_decay":1e-3},
    criterion="cross_entropy",
)
callbacks = [
    cbs.ModelCheckpoint(
        file_name="./bert-small.bin",
        save_dir="./",
        mode="max",
        monitor="val_f1_score",
    ),
    cbs.CosineAnnealingWarmRestarts(T_0=1)
]
metric_list = [SklearnF1()]

In [28]:
bert_exp = Experiment(num_epochs=5, seed=42, fp16=True, device="cuda")

In [29]:
bert_exp.compile_experiment(model_config=config, callbacks=callbacks, metrics=metric_list)

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
bert_exp.fit_loader(train_dl, valid_dl)


Epoch: 1/5
63/63 [=========================]- 3s 42ms/step - val_loss: 0.3443 - val_f1_score: 0.9044

Epoch: 2/5
63/63 [=========================]- 3s 41ms/step - val_loss: 0.4025 - val_f1_score: 0.9259

Epoch: 3/5
63/63 [=========================]- 3s 42ms/step - val_loss: 0.4427 - val_f1_score: 0.9401

Epoch: 4/5
63/63 [=========================]- 3s 42ms/step - val_loss: 0.5744 - val_f1_score: 0.9493

Epoch: 5/5
63/63 [=========================]- 3s 42ms/step - val_loss: 0.4563 - val_f1_score: 0.9560


In [31]:
test_df = pd.read_csv("/content/drive/MyDrive/Hahakathon/gold-test-27446.csv")

In [32]:
test_dl = TextDataloader.from_df(
    df=test_df,
    input_col="text",
    label_cols=None,
    tokenizer=tokenizer,
    max_len=256,
).get_loader(batch_size=16, shuffle = False)

In [36]:
outputs = []
for op in bert_exp.predict_on_loader(test_dl = test_dl, path_to_model= "./bert-small.bin", device = "cuda"):
    outputs.extend(torch.argmax(op , dim = 1))

In [40]:
print(f"Task 1A-Gold-Test(F1-Score) : {skm.f1_score(test_df.is_humor.values , outputs)}")

Task 1A-Gold-Test(F1-Score) : 0.9260143198090692
